Note: The fine-tuning code has been slightly different as the version of openai has been updated to v1.0.0+. 

Here, we provide the latest code.

- Before 2023.11.06, we set ```model = "gpt-3.5-turbo-0613"```
- After 2023.11.06, we can set ```model = "gpt-3.5-turbo-1106" or "gpt-4-0125-preview"```(recommend)

## Environment (OS: Windows or Linux)

In [125]:
!pip install pandas
!pip install openai
!pip install textdistance


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Packages

In [11]:
import pandas as pd
import os
import csv
from openai import OpenAI
from typing import List
import textdistance

## Load Data

## 1. Processing training data into .jsonl

In [6]:
import json
import random
import pandas as pd
from tqdm import tqdm

system_message = """Extract the NMR information from the Paragraph."""

# system_message = """Extract text containing 1H NMR and 13C NMR data, remove interference information such as reactants, raw materials, solvents and other non-final product names based on text semantics, and then extract the name, code or number of the final product. Please delete the IUPAC name Alias, numbers and ordinal numbers before and after fields, such as '2.1.3.', '(HL4)', '(9)', '(4d)'. NMR text should contain complete information, such as instrument power and solvent information, For example, "13C NMR text": "13C NMR (400 MHz, acetone-d6) 174.0 (C), 157.7 (C). Then split the NMR text. The content in NMR conditions is NMR instrument power and solvent information, such as "13C NMR conditions": "400MHz, acetone-d6". The contents in the NMR data are only numbers, such as "13C NMR data": "174.0, 157.7", "1H NMR data": "174.0, 157.7". All responses must originate from information extracted from the given text, ensuring that the extracted content has not been modified or fragmented, and that capitalization and punctuation are exactly the same as the given text. Must end with {"IUPAC":"text","1H NMR text":"text","1H NMR conditions":"text","1H NMR data":"text","13C NMR text":"text","13C NMR conditions":"text","13C NMR data":"text"} format reply."""

def create_user_message(row):
    return f"""{row['Paragraph']}"""

def create_assistant_message(row):
    return f"""{{\"IUPAC\":\"{row['IUPAC']}\",\"1H NMR text\":\"{row['1H NMR text']}\",\"1H NMR conditions\":\"{row['1H NMR conditions']}\",\"1H NMR data\":\"{row['1H NMR data']}\",\"13C NMR text\":\"{row['13C NMR text']}\",\"13C NMR conditions\":\"{row['13C NMR conditions']}\",\"13C NMR data\":\"{row['13C NMR data']}\"}}"""

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    assistant_message = create_assistant_message(row)
    messages.append({"role": "assistant", "content": assistant_message})

    return {"messages": messages}

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)
            
train_input_file_path = "train_25.csv"
train_output_file_path = "train_25.jsonl"
test_input_file_path = "test_300.csv"
prediction_result_path = "prediction_result_without_prompt.csv"

train_df = pd.read_csv(train_input_file_path)
training_data = train_df.apply(prepare_example_conversation, axis=1).tolist()

write_jsonl(training_data, train_output_file_path)

## 2. Upload folds

In [130]:
# It will take few minutes for processing data after uploading ... 

client = OpenAI(
    api_key= 'sk-OPENAI_APIKEY'   # replace with your openai-api-key
)

training_response = client.files.create(
  file = open("train_25.jsonl", "rb"),
  purpose="fine-tune",
)

print("Training file:", training_response.model_dump_json(indent=2))
file_id = training_response.id

Training file: {
  "id": "file-Jzz6hmBZuqLGqrj84tI5Tng5",
  "bytes": 79488,
  "created_at": 1702039513,
  "filename": "train_25_data_in_50.jsonl",
  "object": "file",
  "purpose": "fine-tune",
  "status": "processed",
  "status_details": null
}


## 3. Create a fine-tuning job

- Before 1106, we set ```model = "gpt-3.5-turbo-0613"```

- After 1106, we recommend to set ```model = "gpt-3.5-turbo-1106"``` or ```model = "gpt-3.5-turbo-0125"```

In [131]:
fine_tuning_response = client.fine_tuning.jobs.create(
  training_file = file_id,
  model = "gpt-3.5-turbo-1106",         
  hyperparameters = {"n_epochs":3,}
)

print('fine_tuning_response:', fine_tuning_response.model_dump_json(indent=2))
fine_tuning_job_id = fine_tuning_response.id

fine_tuning_response: {
  "id": "ftjob-krYOi8AUuEO48THQSb7nzy5w",
  "created_at": 1702039515,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "model": "gpt-3.5-turbo-1106",
  "object": "fine_tuning.job",
  "organization_id": "org-4bdp8nLOXTwA9DZ6Apy7enJy",
  "result_files": [],
  "status": "validating_files",
  "trained_tokens": null,
  "training_file": "file-Jzz6hmBZuqLGqrj84tI5Tng5",
  "validation_file": null
}


## 4. Query the state of fine-tuning task 

In [146]:
# run the cell to see the status is "running" or "succeeded"

retrieve_job_response = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
print(retrieve_job_response.model_dump_json(indent=2))
print('status:', retrieve_job_response.status)
fine_tuned_model = retrieve_job_response.fine_tuned_model       # get the fine-tuned model

{
  "id": "ftjob-krYOi8AUuEO48THQSb7nzy5w",
  "created_at": 1702039515,
  "error": null,
  "fine_tuned_model": "ft:gpt-3.5-turbo-1106:personal::8TUbKUIt",
  "finished_at": 1702039833,
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "model": "gpt-3.5-turbo-1106",
  "object": "fine_tuning.job",
  "organization_id": "org-4bdp8nLOXTwA9DZ6Apy7enJy",
  "result_files": [
    "file-1gpirigDitsLdr4MtjlQy3UI"
  ],
  "status": "succeeded",
  "trained_tokens": 97809,
  "training_file": "file-Jzz6hmBZuqLGqrj84tI5Tng5",
  "validation_file": null
}
status: succeeded


## 5. Use the fine-tuend model on single input

In [147]:
system_message = """Extract the NMR information from the Paragraph."""

# system_message = """Extract text containing 1H NMR and 13C NMR data, remove interference information such as reactants, raw materials, solvents and other non-final product names based on text semantics, and then extract the name, code or number of the final product. Please delete the IUPAC name Alias, numbers and ordinal numbers before and after fields, such as '2.1.3.', '(HL4)', '(9)', '(4d)'. NMR text should contain complete information, such as instrument power and solvent information, For example, "13C NMR text": "13C NMR (400 MHz, acetone-d6) 174.0 (C), 157.7 (C). Then split the NMR text. The content in NMR conditions is NMR instrument power and solvent information, such as "13C NMR conditions": "400MHz, acetone-d6". The contents in the NMR data are only numbers, such as "13C NMR data": "174.0, 157.7", "1H NMR data": "174.0, 157.7". All responses must originate from information extracted from the given text, ensuring that the extracted content has not been modified or fragmented, and that capitalization and punctuation are exactly the same as the given text. Must end with {"IUPAC":"text","1H NMR text":"text","1H NMR conditions":"text","1H NMR data":"text","13C NMR text":"text","13C NMR conditions":"text","13C NMR data":"text"} format reply."""

user_message = "3.6.5. 2,3-Di(thio-4-methylphenyl)quinoxaline (2e)Colorless crystals, 1.59 g (85%) yield; m.p. 150 °C. IR (KBr), 3049 (sp2 = C-H), 2968 (sp3-C-H), 1596 (C=N), and 751, 592 (C-S, asymmetric and symmetric stretching) cm−1. 1H-NMR (500 MHz, CDCl3, 300 K), 7.65 (dd, 4H, J = 6.3, 3.5 Hz, H8), 7.53 (d, 4H, J = 8.1 Hz, H2′, and H6′), 7.44 (dd, 2H, J = 6.4, 3.4 Hz, H7), 7.27 (d, 4H, J = 8.0 Hz, H3’and H5’), and 2.42 (s, 6H, 2CH3) ppm. 13C-NMR (APT) (125 MHz, CDCl3, 300 K), δ 154.01 (C), 140.32 (C), 139.26 (C), 135.13 (CH), 129.96 (CH), 128.19 (CH), 127.83 (CH), 124.95 (C), 21.28 (CH3) ppm. C22H18N2S2 requires: C, 70.5%; H, 4.8%; N, 7.4%. Found: C, 70.78%; H, 4.73%; N, 7.68%."

test_messages = []
test_messages.append({"role": "system", "content": system_message})
test_messages.append({"role": "user", "content": user_message})


print("Paragraphs: ", test_messages[1]['content'])

response = client.chat.completions.create(
  model = fine_tuned_model,
  messages = test_messages,
  temperature = 0  
)

print("response: ", response.choices[0].message.content)

Paragraphs:  3.6.5. 2,3-Di(thio-4-methylphenyl)quinoxaline (2e)Colorless crystals, 1.59 g (85%) yield; m.p. 150 °C. IR (KBr), 3049 (sp2 = C-H), 2968 (sp3-C-H), 1596 (C=N), and 751, 592 (C-S, asymmetric and symmetric stretching) cm−1. 1H-NMR (500 MHz, CDCl3, 300 K), 7.65 (dd, 4H, J = 6.3, 3.5 Hz, H8), 7.53 (d, 4H, J = 8.1 Hz, H2′, and H6′), 7.44 (dd, 2H, J = 6.4, 3.4 Hz, H7), 7.27 (d, 4H, J = 8.0 Hz, H3’and H5’), and 2.42 (s, 6H, 2CH3) ppm. 13C-NMR (APT) (125 MHz, CDCl3, 300 K), δ 154.01 (C), 140.32 (C), 139.26 (C), 135.13 (CH), 129.96 (CH), 128.19 (CH), 127.83 (CH), 124.95 (C), 21.28 (CH3) ppm. C22H18N2S2 requires: C, 70.5%; H, 4.8%; N, 7.4%. Found: C, 70.78%; H, 4.73%; N, 7.68%.
response:  {"IUPAC":"2,3-Di(thio-4-methylphenyl)quinoxaline","1H NMR text":"1H-NMR (500 MHz, CDCl3, 300 K), 7.65 (dd, 4H, J = 6.3, 3.5 Hz, H8), 7.53 (d, 4H, J = 8.1 Hz, H2′, and H6′), 7.44 (dd, 2H, J = 6.4, 3.4 Hz, H7), 7.27 (d, 4H, J = 8.0 Hz, H3’and H5’), and 2.42 (s, 6H, 2CH3) ppm","1H NMR conditions":"500 

## 6. Inference on multi inputs

In [149]:
# Here we take 25 of 300 test data for inferencing.
test_df = pd.read_csv(test_input_file_path)
test_paragraphs = list(test_df['Paragraph'][:25])
test_preds = []

total_tokens = 0
for user_message in tqdm(test_paragraphs):
  syst_message = system_message
  test_messages = []
  test_messages.append({"role": "system", "content": syst_message})
  test_messages.append({"role": "user", "content": user_message})

  print("\nParagraphs: ", test_messages[1]['content'])

  response = client.chat.completions.create(
      model = fine_tuned_model,   # model_name_id
      messages = test_messages,   # input
      temperature = 0
  )

  output = response.choices[0].message.content
  print("response: ", output)
  test_preds.append(output)
  tokens_number = response.usage.total_tokens
  total_tokens += tokens_number
# print("used total_tokens: ", total_tokens)

  0%|          | 0/25 [00:00<?, ?it/s]


Paragraphs:  Synthesis and crystallization A stirred solution of 3,5-bis­(tri­fluoro­meth­yl) aceto­phenone (0.5 g, 1.95 mmol) in acetic acid (5 mL) was added dropwise to bromine (0.312 g, 1.95 mmol) in acetic acid. The reaction medium was stirred at room temperature for 5 h. To the resulting mixture, water (5 mL) was added and the mixture was concentrated under reduced pressure. The residue obtained was diluted with ethyl­acetate (10 mL), the organic layer washed with water (10 mL) and a sodium bicarbonate solution (5 mL), and filtered through dried sodium sulfate and evaporated to obtain 1-(3,5-bis­(tri­fluoro­meth­yl)phen­yl)-2-bromo­ethanone as a light-yellow solid in 62% yield. m.p: 317–318 K. 1H NMR: (CDCl3, 600 MHz): 8.44 (2H, s), 8.13 (1H, s), 4.48 (2H, s); 13C NMR: (CDCl3, 150 MHz): 188.81, 135.31, 133.06, 132.83, 132.60, 128.99, 127.08, 127.06, 125.42, 123.61, 121.80, 120.00, 29.46.  Refinement Crystal data, data collection and structure refinement details are summarized in 

  4%|▍         | 1/25 [00:05<02:09,  5.41s/it]

response:  {"IUPAC":"1-(3,5-bis­(tri­fluoro­meth­yl)phen­yl)-2-bromo­ethanone","1H NMR text":"1H NMR: (CDCl3, 600 MHz): 8.44 (2H, s), 8.13 (1H, s), 4.48 (2H, s)","1H NMR conditions":"CDCl3, 600 MHz","1H NMR data":"8.44, 8.13, 4.48","13C NMR text":"13C NMR: (CDCl3, 150 MHz): 188.81, 135.31, 133.06, 132.83, 132.60, 128.99, 127.08, 127.06, 125.42, 123.61, 121.80, 120.00, 29.46","13C NMR conditions":"CDCl3, 150 MHz","13C NMR data":"188.81, 135.31, 133.06, 132.83, 132.60, 128.99, 127.08, 127.06, 125.42, 123.61, 121.80, 120.00, 29.46"}

Paragraphs:  Synthesis of N-(diaminomethylidene)-4-[(E)-(4-hydroxyphenyl)diazenyl] benzenesulfonamide (SPh) A solution of sodium nitrite (0.17 g, 7.3 mmol) in water (2 mL) was slowly added to an acidified solution of sulfaguanidine (0.64 g, 3 mmol) in water/acetone mixture (12 mL, 2:1) at 0°C. The resulting diazonium salt solution was gently added to an ice-cold solution of phenol (0.29 g, 3 mmol), NaOH (0.34 g, 8.5 mmol) and Na2CO3 (0.9 g, 8.5 mmol) in 10 mL

  8%|▊         | 2/25 [00:08<01:35,  4.16s/it]

...

response:  {"IUPAC":"7-(9H-Carbazol-9-yl)heptane-1-thiol","1H NMR text":"1H NMR (400 MHz, CDCl3) δ: 8.14 (d, J = 7.6 Hz, 2H, Ar), 7.51–7.48 (m, 2H, Ar), 7.43 (d, J = 8.1 Hz, 2H, Ar), 7.27 (m, 2H, Ar), 4.33–4.31 (m, 2H, NCH2), 2.66–2.63 (m, 2H, CH2S), 1.92–1.86 (m, 2H, NCH2CH2), 1.66–1.64 (m, 2H, HSCH2CH2), 1.41–1.37 (m, 7H, 3CH2, SH) ppm","1H NMR conditions":"400 MHz, CDCl3","1H NMR data":"8.14, 7.51–7.48, 7.43, 7.27, 4.33–4.31, 2.66–2.63, 1.92–1.86, 1.66–1.64, 1.41–1.37","13C NMR text":"13C NMR (100 MHz, CDCl3) δ: 140.73, 126.45, 124.15, 121.68, 120.48, 109.63, 47.43, 33.25, 28.96, 28.61, 27.95, 27.55, 25.38 ppm","13C NMR conditions":"100 MHz, CDCl3","13C NMR data":"140.73, 126.45, 124.15, 121.68, 120.48, 109.63, 47.43, 33.25, 28.96, 28.61, 27.95, 27.55, 25.38"}

Paragraphs:  3-(3-Methoxyphenyl)benzo[4,5]imidazo[2,1-b]thiazole (4a)White crystals (yield 80%), m.p. 173–175 °C; 1H NMR (DMSO-d6) δ ppm: 3.84 (s, 3H, OCH3), 7.13 (d, 1H, Ar–H, J = 7.5 Hz), 7.20–7.32 (m, 6H, Ar–H), 7.54 (t, 1

 96%|█████████▌| 24/25 [02:23<00:05,  5.14s/it]

response:  {"IUPAC":"3-(3-Methoxyphenyl)benzo[4,5]imidazo[2,1-b]thiazole","1H NMR text":"1H NMR (DMSO-d6) δ ppm: 3.84 (s, 3H, OCH3), 7.13 (d, 1H, Ar–H, J = 7.5 Hz), 7.20–7.32 (m, 6H, Ar–H), 7.54 (t, 1H, Ar–H, J = 8.0 Hz), 7.71 (d, 1H, Ar–H, J = 8.0 Hz)","1H NMR conditions":"DMSO-d6","1H NMR data":"3.84, 7.13, 7.20–7.32, 7.54, 7.71","13C NMR text":"13C NMR (DMSO-d6) δ ppm: 55.86 (OCH3), 109.23, 112.01, 114.68, 116.44, 119.23, 120.86, 121.47, 123.61, 130.61, 130.69, 133.66, 148.64, 157.03, 159.92","13C NMR conditions":"DMSO-d6","13C NMR data":"55.86, 109.23, 112.01, 114.68, 116.44, 119.23, 120.86, 121.47, 123.61, 130.61, 130.69, 133.66, 148.64, 157.03, 159.92"}

Paragraphs:   2-(1H-imidazol-4-yl)-N-(4-methylbenzylidene)ethanamine (H3): Yield: 72%; colour: white powder, mp: 190–193 °C; FT-IR (cm−1): 1640 (–C=N–); 1H NMR (DMSO-d6, 300 MHz, δ ppm): 8.62 (s, 1H, –N=CH–), 7.88 (d, 1H, J = 1.0, H-2 Im), 7.78 (d, 2H, J = 8.2, Ar-H), 7.35 (s, 1H, H-5 Im), 7.28 (d, 2H, J = 8.2, Ar-H), 3.65 (t, 2H

100%|██████████| 25/25 [02:27<00:00,  5.92s/it]

response:  {"IUPAC":"2-(1H-imidazol-4-yl)-N-(4-methylbenzylidene)ethanamine","1H NMR text":"1H NMR (DMSO-d6, 300 MHz, δ ppm): 8.62 (s, 1H, –N=CH–), 7.88 (d, 1H, J = 1.0, H-2 Im), 7.78 (d, 2H, J = 8.2, Ar-H), 7.35 (s, 1H, H-5 Im), 7.28 (d, 2H, J = 8.2, Ar-H), 3.65 (t, 2H, J = 6.2, –CH2CH2-Im), 2.90 (t, 2H, J = 6.2, –CH2CH2-Im), 1.85 (s, 3H, –CH3)","1H NMR conditions":"DMSO-d6, 300 MHz, δ ppm","1H NMR data":"8.62, 7.88, 7.78, 7.35, 7.28, 3.65, 2.90, 1.85","13C NMR text":"13C NMR (DMSO-d6, 75 MHz, δ ppm): 166.41 (–N=CH–), 163.82, 133.51, 132.93, 131.65, 116.31, 115.64, 114.73, 111.42, 55.24, 26.72, 18.38","13C NMR conditions":"DMSO-d6, 75 MHz, δ ppm","13C NMR data":"166.41, 163.82, 133.51, 132.93, 131.65, 116.31, 115.64, 114.73, 111.42, 55.24, 26.72, 18.38"}


# 7. Save results

In [ ]:
# Split the dict into dataframe (.csv)

csv_fp = open(prediction_result_path, "a+", encoding='utf-8', newline='')
writer = csv.writer(csv_fp)
writer.writerow(['IUPAC', '1H NMR text', '1H NMR conditions', '1H NMR data', '13C NMR text', '13C NMR conditions', '13C NMR data'])

for row in test_preds:
    predicion = json.loads(row)
    sheet_IUPAC = predicion['IUPAC']
    sheet_1H_NMR_text = predicion['1H NMR text']
    sheet_1H_NMR_conditions = predicion['1H NMR conditions']
    sheet_1H_NMR_data = predicion['1H NMR data']
    sheet_13C_NMR_text = predicion['13C NMR text']
    sheet_13C_NMR_conditions = predicion['13C NMR conditions']
    sheet_13C_NMR_data = predicion['13C NMR data']
    writer.writerow([sheet_IUPAC, 
                     sheet_1H_NMR_text, sheet_1H_NMR_conditions, sheet_1H_NMR_data, 
                     sheet_13C_NMR_text, sheet_13C_NMR_conditions, sheet_13C_NMR_data])
csv_fp.close()

# 8. Evaluate

In [12]:
ground_truth_IUPAC = []
ground_truth_1H_NMR = []
ground_truth_1H_NMR_solvent = []
ground_truth_1H_NMR_data = []
ground_truth_13C_NMR = []
ground_truth_13C_NMR_solvent = []
ground_truth_13C_NMR_data = []

predict_IUPAC = []
predict_1HNMR = []
predict_1HNMR_solvent = []
predict_1HNMR_data = []
predict_13CNMR = []
predict_13CNMR_solvent = []
predict_13CNMR_data = []

### Task 4: Paragraph2NMR

# ground truth
ground_truth = pd.read_csv(r'test_300.csv',encoding='utf-8')
ground_truth = ground_truth[:25].fillna('N/A')

for i,row in ground_truth.iterrows():
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_IUPAC.append(row['IUPAC'])
    ground_truth_1H_NMR.append(row['1H NMR text'])
    ground_truth_1H_NMR_solvent.append(row['1H NMR conditions'])
    ground_truth_1H_NMR_data.append(row['1H NMR data'])
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_13C_NMR_solvent.append(row['13C NMR conditions'])
    ground_truth_13C_NMR_data.append(row['13C NMR data'])

# prediction
prediction_df = pd.read_csv(prediction_result_path) # 
prediction_df = prediction_df.fillna('N/A')

for i,row in prediction_df.iterrows():
    predict_13CNMR.append(row['13C NMR text'])
    predict_IUPAC.append(row['IUPAC'])
    predict_1HNMR.append(row['1H NMR text'])
    predict_1HNMR_solvent.append(row['1H NMR conditions'])
    predict_1HNMR_data.append(row['1H NMR data'])
    predict_13CNMR.append(row['13C NMR text'])
    predict_13CNMR_solvent.append(row['13C NMR conditions'])
    predict_13CNMR_data.append(row['13C NMR data'])

def levenshtein_similarity(truth: List[str], pred: List[str]) -> float:
    assert len(truth) == len(pred)
    scores = sum(textdistance.levenshtein.normalized_similarity(t, p) for t, p in zip(truth, pred))
    return scores / len(truth)

def full_sentence_accuracy(truth: List[str], pred: List[str]) -> float:
    """Calculate the number of exact matches."""
    assert len(truth) == len(pred)
    correct_count = sum(int(t == p) for t, p in zip(truth, pred))
    return correct_count / len(truth)

# Evaluation
print("levenshtein_similarity_of_IUPAC: ", levenshtein_similarity(ground_truth_IUPAC, predict_IUPAC))
print("levenshtein_similarity_of_1HNMR_text: ", levenshtein_similarity(ground_truth_1H_NMR, predict_1HNMR))
print("levenshtein_similarity_of_1HNMR_conditions: ", levenshtein_similarity(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("levenshtein_similarity_of_1HNMR_data: ", levenshtein_similarity(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("levenshtein_similarity_of_13CNMR_text: ", levenshtein_similarity(ground_truth_13C_NMR, predict_13CNMR))
print("levenshtein_similarity_of_13CNMR_conditions: ", levenshtein_similarity(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("levenshtein_similarity_of_13CNMR_data: ", levenshtein_similarity(ground_truth_13C_NMR_data, predict_13CNMR_data))
print()
print("full_sentence_accuracy_of_IUPAC: ", full_sentence_accuracy(ground_truth_IUPAC, predict_IUPAC))
print("full_sentence_accuracy_of_1HNMR_text: ", full_sentence_accuracy(ground_truth_1H_NMR, predict_1HNMR))
print("full_sentence_accuracy_of_1HNMR_conditions: ", full_sentence_accuracy(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("full_sentence_accuracy_of_1HNMR_data: ", full_sentence_accuracy(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("full_sentence_accuracy_of_13CNMR_text: ", full_sentence_accuracy(ground_truth_13C_NMR, predict_13CNMR))
print("full_sentence_accuracy_of_13CNMR_conditions: ", full_sentence_accuracy(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("full_sentence_accuracy_of_13CNMR_data: ", full_sentence_accuracy(ground_truth_13C_NMR_data, predict_13CNMR_data))

levenshtein_similarity_of_IUPAC:  0.9155886049434437
levenshtein_similarity_of_1HNMR_text:  0.989070677192862
levenshtein_similarity_of_1HNMR_conditions:  0.9706946778711485
levenshtein_similarity_of_1HNMR_data:  0.9975917312661499
levenshtein_similarity_of_13CNMR_text:  0.9962408081442666
levenshtein_similarity_of_13CNMR_conditions:  0.9359047619047618
levenshtein_similarity_of_13CNMR_data:  0.9987763007907716

full_sentence_accuracy_of_IUPAC:  0.84
full_sentence_accuracy_of_1HNMR_text:  0.52
full_sentence_accuracy_of_1HNMR_conditions:  0.56
full_sentence_accuracy_of_1HNMR_data:  0.92
full_sentence_accuracy_of_13CNMR_text:  0.6
full_sentence_accuracy_of_13CNMR_conditions:  0.6
full_sentence_accuracy_of_13CNMR_data:  0.88
